In [47]:
import numpy as np

In [ ]:
def fit(x_train,y_train):
    result={}
    class_values=set(y_train)
    for current_class in class_values:
        num_features=x_train.shape[1]
        result[current_class]={}
        result['total_data']=len(y_train)
        current_class_rows=(y_train==current_class)
        x_train_current=x_train[current_class_rows]
        y_train_current=y_train[current_class_rows]
        result[current_class]['total_count']=len(y_train_current)
        for j in range(1,num_features+1):
            result[current_class][j]={}
            allpossible_values=set(x_train[:,j-1])
            for value in allpossible_values:
                result[current_class][j][value]=(x_train_current[:,j-1]==value).sum()
    return result

In [53]:
def probablity(dictionary,x,current_class):
    output=np.log(dictionary[current_class]['total_count'])-np.log(dictionary['total_data'])
    num_features=len(dictionary[current_class].keys())-1
    for j in range(1,num_features+1):
        xj=x[j-1]
        count_current_class_with_value_xj=dictionary[current_class][j][xj]
        count_current_class=dictionary[current_class]['total_count']
        current_xj_probablity=np.log(count_current_class_with_value_xj)-np.log(count_current_class)
        output=output+current_xj_probablity
    return output

In [11]:
def predictsinglepoint(dictionary,x):
    classes=dictionary.keys()
    best_p=-1000
    best_class=-1
    first_run=True
    for current_class in classes:
        if(current_class=='total_data'):
            continue
        p_current_class=probablity(dictionary,x,current_class)
        if(first_run or best_p < p_current_class):
            best_p=p_current_class
            best_class=current_class
        first_run=False
    return best_class

In [12]:
def predict(dictionary,x_test):
    y_pred=[]
    for x in x_test:
        x_class=predictsinglepoint(dictionary,x)
        y_pred.append(x_class)
    return y_pred

        

In [5]:
def makelabeled(column):
    second_limit=column.mean()
    first_limit=0.5*second_limit
    third_limit=1.5*second_limit
    for i in range(0,len(column)):
        if(column[i]<first_limit):
            column[i]=0
        elif(column[i]<second_limit):
            column[i]=1
        elif(column[i]<third_limit):
            column[i]=2
        else:
            column[i]=3
    return column

In [6]:
from sklearn import datasets
iris=datasets.load_iris()
x=iris.data
y=iris.target
for i in range(x.shape[1]):
    x[:,i]=makelabeled(x[:,i])

In [44]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [45]:
dictionary=fit(x_train,y_train)

In [54]:
y_pred=predict(dictionary,x_test)

C:\Users\ansh41\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [56]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       0.93      1.00      0.96        13
          2       1.00      0.92      0.96        12

avg / total       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 13  0]
 [ 0  1 11]]
